In [1]:
# Import the modules
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.metrics import balanced_accuracy_score, confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import StandardScaler
import hvplot.pandas

ImportError: DLL load failed: The specified module could not be found.

---

## Split the Data into Training and Testing Sets

### Step 1: Read the `lending_data.csv` data from the `Resources` folder into a Pandas DataFrame.

In [ ]:
# Read the CSV file from the Resources folder into a Pandas DataFrame
data = Path('Resources/lending_data.csv')
df = pd.read_csv(data)

# Review the DataFrame
df.head()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
0,10700.0,7.672,52800,0.431818,5,1,22800,0
1,8400.0,6.692,43600,0.311927,3,0,13600,0
2,9000.0,6.963,46100,0.349241,3,0,16100,0
3,10700.0,7.664,52700,0.430740,5,1,22700,0
4,10800.0,7.698,53000,0.433962,5,1,23000,0


In [ ]:
# Generate summary stats
df.describe()

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt,loan_status
count,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000,77536.000000
mean,9805.562577,7.292333,49221.949804,0.377318,3.826610,0.392308,19221.949804,0.032243
std,2093.223153,0.889495,8371.635077,0.081519,1.904426,0.582086,8371.635077,0.176646
min,5000.000000,5.250000,30000.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8700.000000,6.825000,44800.000000,0.330357,3.000000,0.000000,14800.000000,0.000000
50%,9500.000000,7.172000,48100.000000,0.376299,4.000000,0.000000,18100.000000,0.000000
75%,10400.000000,7.528000,51400.000000,0.416342,4.000000,1.000000,21400.000000,0.000000
max,23800.000000,13.235000,105200.000000,0.714829,16.000000,3.000000,75200.000000,1.000000


### Step 2: Create the labels set (`y`)  from the “loan_status” column, and then create the features (`X`) DataFrame from the remaining columns.

In [ ]:
# Separate the data into labels and features
y = df["loan_status"] #label
X = df.drop(columns="loan_status") #features


In [ ]:
# Review the y variable Series to see if it contain data
y[:5]

0    0
1    0
2    0
3    0
4    0
Name: loan_status, dtype: int64

In [ ]:
# Review the X variable DataFrame
X[:5]

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,10700.0,7.672,52800,0.431818,5,1,22800
1,8400.0,6.692,43600,0.311927,3,0,13600
2,9000.0,6.963,46100,0.349241,3,0,16100
3,10700.0,7.664,52700,0.430740,5,1,22700
4,10800.0,7.698,53000,0.433962,5,1,23000


### Step 3: Check the balance of the labels variable (`y`) by using the `value_counts` function.

In [ ]:
# Check the balance of our target values
y.value_counts()


0    75036
1     2500
Name: loan_status, dtype: int64

### Step 4: Split the data into training and testing datasets by using `train_test_split`.

In [ ]:
# Split the data using train_test_split and
# assign a random_state of 1 to the function
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)


---

## Create a Logistic Regression Model with the Original Data

###  Step 1: Fit a logistic regression model by using the training data (`X_train` and `y_train`).

In [ ]:
# Instantiate the Logistic Regression model- assign random_state =1
regression = LogisticRegression(solver= 'lbfgs', random_state=1)

# Fit the model using training data
regression.fit(X_train, y_train)


LogisticRegression(random_state=1)

### Step 2: Save the predictions on the testing data labels by using the testing feature data (`X_test`) and the fitted model.

In [ ]:
# Make a prediction using the testing data
y_prediction = regression.predict(X_test)
y_prediction[:5]

array([0, 0, 0, 0, 0], dtype=int64)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [ ]:
# Print the balanced_accuracy score of the model
y_pred_accur_score= balanced_accuracy_score(y_test, y_prediction)
print(y_pred_accur_score)


0.9520479254722232


In [ ]:
# Generate a confusion matrix for the model
matrix = confusion_matrix(y_test, y_prediction)
print(matrix)

[[18663   102]
 [   56   563]]


In [ ]:
# Print the classification report for the model
report = classification_report(y_test, y_prediction)
print(report)

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.85      0.91      0.88       619

    accuracy                           0.99     19384
   macro avg       0.92      0.95      0.94     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question.

**Question:** How well does the logistic regression model predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** At 0.99 accuracy, the linear regression model demonstrates to be an accurate model when predicting high-risk loans; balanced accuracy 0.95. The classification report looks excellent at 1.00 precision and 0.99 recall. In sum, the model appears to be an excellent predictor of high-risk loan categories.


---

## Predict a Logistic Regression Model with Resampled Training Data

### Step 1: Use the `RandomOverSampler` module from the imbalanced-learn library to resample the data. Be sure to confirm that the labels have an equal number of data points. 

In [ ]:
# Instantiate the random oversampler model, random_state = 1 
oversampler = RandomOverSampler(random_state=1)

# Fit the original training data to the random_oversampler model
X_resampled, y_resampled = oversampler.fit_resample(X_train, y_train)

X_resampled[:5]

,loan_size,interest_rate,borrower_income,debt_to_income,num_of_accounts,derogatory_marks,total_debt
0,8600.0,6.792,44500,0.325843,3,0,14500
1,7800.0,6.419,41000,0.268293,2,0,11000
2,10000.0,7.386,50100,0.401198,4,1,20100
3,9300.0,7.093,47300,0.365751,3,0,17300
4,9200.0,7.045,46900,0.360341,3,0,16900


In [ ]:
# make prediction X_test data
prediction = regression.predict(X_test)
prediction[:5]

array([0, 0, 0, 0, 0], dtype=int64)

In [ ]:
#Find balanced accruacy score
accuracy_score= balanced_accuracy_score(y_test, prediction)
print(accuracy_score)

0.9520479254722232


In [ ]:
# Count the distinct values of the resampled labels data to see if the data is balanced
unique_value, counts = np.unique(y_resampled, return_counts=True)
print(unique_value, counts)

[0 1] [56271 56271]


### Step 2: Use the `LogisticRegression` classifier and the resampled data to fit the model and make predictions.

In [ ]:
# Instantiate the Logistic Regression model, random_state = 1
model = LogisticRegression(solver= 'lbfgs', random_state=1)

# Fit the model using training data
model.fit(X_resampled, y_resampled)

# Make a prediction using the testing data
prediction = model.predict(X_test)

prediction[:5]


array([0, 0, 0, 0, 0], dtype=int64)

### Step 3: Evaluate the model’s performance by doing the following:

* Calculate the accuracy score of the model.

* Generate a confusion matrix.

* Print the classification report.

In [ ]:
# Print the balanced_accuracy score of the model 
balanced_accuracy = balanced_accuracy_score(y_test, prediction)
print(balanced_accuracy)


0.9936781215845847


In [ ]:
# Generate a confusion matrix for the model
matrix =confusion_matrix(y_test, prediction)
print(matrix)

[[18649   116]
 [    4   615]]


In [ ]:
# Print the classification report for the model
report = classification_report(y_test, prediction)
print(report)

              precision    recall  f1-score   support

           0       1.00      0.99      1.00     18765
           1       0.84      0.99      0.91       619

    accuracy                           0.99     19384
   macro avg       0.92      0.99      0.95     19384
weighted avg       0.99      0.99      0.99     19384



### Step 4: Answer the following question

 **Question:** How well does the logistic regression model, fit with oversampled data, predict both the `0` (healthy loan) and `1` (high-risk loan) labels?

**Answer:** The values of accuracy, precision, and recall are from 0.99 to 1.00; the balanaced accuracy rate was 0.95. This suggests that, after balancing the data sets, the model is near perfect in identifying high-risk loan labels, resulting in a robust tool to minimize loan risk. These results were identical to the previous linear regression, without balancing the data. When comparing the respective matrices of confusion, there is some variation that might be used to make a more definitive conclusion as to which model would be preferred, based on rates of false positive or false negatives.